In [170]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from datetime import date
from pyspark.sql import Window, Row
from user_agents import parse

# Создание SparkSession

In [171]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

Создание дата фрейма лог файла

In [172]:
schema = T.StructType(fields=[
    T.StructField("IP", T.StringType(), True),
    T.StructField("sign_1", T.StringType(), True),
    T.StructField("sign_2", T.StringType(), True),
    T.StructField("Date_access", T.StringType(), True),
    T.StructField("Date_access_", T.StringType(), True),
    T.StructField("Action", T.StringType (), True),
    T.StructField("Status", T.IntegerType (), True),
    T.StructField("Size", T.IntegerType (), True),
    T.StructField("sign_3", T.StringType (), True),
    T.StructField("User_agent", T.StringType (), True),
    T.StructField("sign_4", T.StringType (), True)
])

In [173]:
path = "data/access.log"
df1 = spark.read.csv(path, schema=schema, sep=" ")
df1.show(10)

+-------------+------+------+--------------------+------------+--------------------+------+-----+--------------------+--------------------+------+
|           IP|sign_1|sign_2|         Date_access|Date_access_|              Action|Status| Size|              sign_3|          User_agent|sign_4|
+-------------+------+------+--------------------+------------+--------------------+------+-----+--------------------+--------------------+------+
| 54.36.149.41|     -|     -|[22/Jan/2019:03:5...|      +0330]|GET /filter/27|13...|   200|30577|                   -|Mozilla/5.0 (comp...|     -|
|  31.56.96.51|     -|     -|[22/Jan/2019:03:5...|      +0330]|GET /image/60844/...|   200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|     -|
|  31.56.96.51|     -|     -|[22/Jan/2019:03:5...|      +0330]|GET /image/61474/...|   200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|     -|
|40.77.167.129|     -|     -|[22/Jan/2019:03:5...|      +0330]|GET /image/14925/...|   200| 1696|                   -|

Создание дата фрема файла csv

In [174]:
schema_cl = T.StructType(fields=[
    T.StructField("client", T.StringType(), True),
    T.StructField("hostname", T.StringType(), True),
    T.StructField("alias_list", T.StringType(), True),
    T.StructField("address_list", T.StringType(), True)
])

In [175]:
path = "data/client_.txt"
df2 = spark.read.csv(path, schema=schema_cl, sep=",")
df2.show(10)

+-------------+--------------------+--------------------+-----------------+
|       client|            hostname|          alias_list|     address_list|
+-------------+--------------------+--------------------+-----------------+
| 5.123.144.95|        5.123.144.95|[Errno 1] Unknown...|             null|
| 5.122.76.187|        5.122.76.187|[Errno 1] Unknown...|             null|
| 5.215.249.99|        5.215.249.99|[Errno 1] Unknown...|             null|
|31.56.102.211|31-56-102-211.sha...|['211.102.56.31.i...|['31.56.102.211']|
|5.123.166.223|       5.123.166.223|[Errno 1] Unknown...|             null|
|  5.160.26.98|         5.160.26.98|[Errno 1] Unknown...|             null|
|5.127.147.132|       5.127.147.132|[Errno 1] Unknown...|             null|
|158.58.30.218|       158.58.30.218|[Errno 1] Unknown...|             null|
| 86.55.230.86|        86.55.230.86|[Errno 1] Unknown...|             null|
| 89.35.65.186|        89.35.65.186|[Errno 1] Unknown...|             null|
+-----------

# Проверка содержимого столбцов

In [176]:
df1.count()

100

In [177]:
df1.groupby("IP")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = True)\
   .show()

+--------------+-----+
|            IP|activ|
+--------------+-----+
|   5.211.97.39|    1|
|  17.58.102.43|    1|
| 34.247.132.53|    1|
|89.199.193.251|    1|
|  54.36.149.70|    1|
|  54.36.149.41|    1|
|  54.36.148.87|    1|
|  5.112.52.254|    2|
|  2.179.141.98|    2|
| 207.46.13.136|    3|
| 173.249.54.67|    4|
|   91.99.72.15|    6|
|  66.249.66.91|    6|
| 178.253.33.51|    6|
| 207.46.13.115|    7|
|   31.56.96.51|    7|
| 66.111.54.249|    7|
| 66.249.66.194|    9|
| 40.77.167.129|   10|
|   5.78.198.52|   10|
+--------------+-----+
only showing top 20 rows



In [178]:
 df1.groupby("IP")\
    .agg(F.count("*").alias("activ"))\
    .orderBy("activ", ascending = False)\
    .agg(F.sum("activ"))\
    .show()

+----------+
|sum(activ)|
+----------+
|       100|
+----------+



In [179]:
df1.groupby("sign_1")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+------+-----+
|sign_1|activ|
+------+-----+
|     -|  100|
+------+-----+



In [180]:
df1.groupby("sign_2")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+------+-----+
|sign_2|activ|
+------+-----+
|     -|  100|
+------+-----+



In [181]:
df1.groupby("sign_3")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+--------------------+-----+
|              sign_3|activ|
+--------------------+-----+
|                   -|   53|
|https://www.zanbi...|   14|
|https://www.zanbi...|    9|
|https://www.zanbi...|    7|
|https://www.zanbi...|    6|
|https://www.zanbi...|    5|
|https://www.zanbi...|    2|
|https://www.zanbi...|    1|
|https://www.zanbi...|    1|
|https://www.zanbi...|    1|
|https://www-zanbi...|    1|
+--------------------+-----+



In [182]:
df1.groupby("sign_3")\
    .agg(F.count("*").alias("activ"))\
    .orderBy("activ", ascending = False)\
    .agg(F.sum("activ"))\
    .show()

+----------+
|sum(activ)|
+----------+
|       100|
+----------+



In [183]:
df1.groupby("sign_4")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+------+-----+
|sign_4|activ|
+------+-----+
|     -|  100|
+------+-----+



In [184]:
df1.groupby("Date_access_")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = False)\
   .show()

+------------+-----+
|Date_access_|activ|
+------------+-----+
|      +0330]|  100|
+------------+-----+



Убираем неинформативные столбцы

In [140]:
df1_2 = df1.drop("sign_1", "sign_2", "Date_access_", "sign_3", "sign_4")
df1_2.show()

+-------------+--------------------+--------------------+------+-----+--------------------+
|           IP|         Date_access|              Action|Status| Size|          User_agent|
+-------------+--------------------+--------------------+------+-----+--------------------+
| 54.36.149.41|[22/Jan/2019:03:5...|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/60844/...|   200| 5667|Mozilla/5.0 (Linu...|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/61474/...|   200| 5379|Mozilla/5.0 (Linu...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/14925/...|   200| 1696|Mozilla/5.0 (comp...|
|  91.99.72.15|[22/Jan/2019:03:5...|GET /product/3189...|   200|41483|Mozilla/5.0 (Wind...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/23488/...|   200| 2654|Mozilla/5.0 (comp...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/45437/...|   200| 3688|Mozilla/5.0 (comp...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/576/ar...|   200|14776|Mozilla/5.

Анализ User_agent для выделения данных по устройству и по браузеру

In [187]:
#https://pypi.org/project/user-agents/
n = 100
ls = [[],[]]
for i in range(n):
    ua_string = str(df1_2.select(["User_agent"]).collect()[i])
    user_agent = parse(ua_string)
    ls.append([user_agent.browser.family, user_agent.device.brand])
        
ls

[[],
 [],
 ['AhrefsBot', 'Spider'],
 ['Chrome Mobile', 'Huawei'],
 ['Chrome Mobile', 'Huawei'],
 ['bingbot', 'Spider'],
 ['Firefox', None],
 ['bingbot', 'Spider'],
 ['bingbot', 'Spider'],
 ['bingbot', 'Spider'],
 ['Googlebot', 'Spider'],
 ['bingbot', 'Spider'],
 ['bingbot', 'Spider'],
 ['bingbot', 'Spider'],
 ['Chrome Mobile', 'HTC'],
 ['bingbot', 'Spider'],
 ['Chrome', None],
 ['bingbot', 'Spider'],
 ['bingbot', 'Spider'],
 ['bingbot', 'Spider'],
 ['bingbot', 'Spider'],
 ['Googlebot', 'Spider'],
 ['Chrome', None],
 ['Chrome Mobile', 'HTC'],
 ['Googlebot', 'Spider'],
 ['Chrome Mobile', 'Huawei'],
 ['Chrome Mobile', 'Huawei'],
 ['Chrome Mobile', 'HTC'],
 ['Samsung Internet', 'Samsung'],
 ['Chrome Mobile', 'HTC'],
 ['bingbot', 'Spider'],
 ['Chrome', None],
 ['Samsung Internet', 'Samsung'],
 ['Googlebot', 'Spider'],
 ['Googlebot', 'Spider'],
 ['Chrome', None],
 ['AhrefsBot', 'Spider'],
 ['Firefox Mobile', 'Generic'],
 ['Firefox Mobile', 'Generic'],
 ['Firefox Mobile', 'Generic'],
 ['Chrom

In [188]:
del ls[0]
del ls[0]

In [189]:
len(ls)

100

In [190]:
R = Row('Browser', 'Name_Device')

df1_3 = spark.createDataFrame([R(x, y) for x, y in ls])
df1_3.show() 

+-------------+-----------+
|      Browser|Name_Device|
+-------------+-----------+
|    AhrefsBot|     Spider|
|Chrome Mobile|     Huawei|
|Chrome Mobile|     Huawei|
|      bingbot|     Spider|
|      Firefox|       null|
|      bingbot|     Spider|
|      bingbot|     Spider|
|      bingbot|     Spider|
|    Googlebot|     Spider|
|      bingbot|     Spider|
|      bingbot|     Spider|
|      bingbot|     Spider|
|Chrome Mobile|        HTC|
|      bingbot|     Spider|
|       Chrome|       null|
|      bingbot|     Spider|
|      bingbot|     Spider|
|      bingbot|     Spider|
|      bingbot|     Spider|
|    Googlebot|     Spider|
+-------------+-----------+
only showing top 20 rows



Формирование базовой таблицы

In [199]:
df1_4 = df1_2.join(df1_3)
df1_4.show() 

+-------------+--------------------+--------------------+------+-----+--------------------+---------+-----------+
|           IP|         Date_access|              Action|Status| Size|          User_agent|  Browser|Name_Device|
+-------------+--------------------+--------------------+------+-----+--------------------+---------+-----------+
| 54.36.149.41|[22/Jan/2019:03:5...|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|AhrefsBot|     Spider|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/60844/...|   200| 5667|Mozilla/5.0 (Linu...|AhrefsBot|     Spider|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/61474/...|   200| 5379|Mozilla/5.0 (Linu...|AhrefsBot|     Spider|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/14925/...|   200| 1696|Mozilla/5.0 (comp...|AhrefsBot|     Spider|
|  91.99.72.15|[22/Jan/2019:03:5...|GET /product/3189...|   200|41483|Mozilla/5.0 (Wind...|AhrefsBot|     Spider|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/23488/...|   200| 2654|Mozilla/5.0 (comp.

In [ ]:
Формирование таблицы "Устройства по пользователям"

In [241]:
df_Devices_ = df1_4.groupby('Name_Device')\
             .agg(F.count('IP').alias('Count_Users'))\
             .withColumn('Count_Users',F.col('Count_Users'))
             
df_Devices_.show()

+-----------+-----------+
|Name_Device|Count_Users|
+-----------+-----------+
|     Spider|       3800|
|     Huawei|       1100|
|       null|        900|
|    Samsung|       1900|
|        HTC|        600|
|      Apple|        300|
|    Generic|       1400|
+-----------+-----------+



In [230]:
#df_Devices = df_Devices.na.fill("other")

In [258]:
df_D2 = df_Devices_.filter(df_Devices_.Name_Device != "null")
df_D2.show()

+-----------+-----------+
|Name_Device|Count_Users|
+-----------+-----------+
|     Spider|       3800|
|     Huawei|       1100|
|    Samsung|       1900|
|        HTC|        600|
|      Apple|        300|
|    Generic|       1400|
+-----------+-----------+



In [259]:
df_Devices_Users = df_D2.withColumn('All_Users', F.sum('Count_Users').over(Window.partitionBy()))\
            .withColumn('Ratio_Users', F.col('Count_Users') / F.col('All_Users'))\
            .select('Name_Device', 'Count_Users', 'Ratio_Users')
       
df_Devices_Users.show()


22/12/30 12:33:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-----------+-------------------+
|Name_Device|Count_Users|        Ratio_Users|
+-----------+-----------+-------------------+
|     Spider|       3800| 0.4175824175824176|
|    Samsung|       1900| 0.2087912087912088|
|        HTC|        600|0.06593406593406594|
|     Huawei|       1100|0.12087912087912088|
|      Apple|        300|0.03296703296703297|
|    Generic|       1400|0.15384615384615385|
+-----------+-----------+-------------------+



In [ ]:
Формирование таблицы "Устройства по действиям"

In [261]:
df_D3 = df_Devices_Users.join(df1_4, df_Devices_Users.Name_Device == df1_4.Name_Device)\
     .groupby(df1_4.Name_Device)\
     .agg(F.count(df1_4.Action).alias('Count_Actions'))\
     .withColumn('Count_Actions', F.col('Count_Actions'))
     #.select(df_D2.Name_Device, df_D2.Count_Users, df_D2.Ratio_Users, df_D2.Count_Actions)
df_D3.show() 

+-----------+-------------+
|Name_Device|Count_Actions|
+-----------+-------------+
|     Spider|         3800|
|     Huawei|         1100|
|    Samsung|         1900|
|        HTC|          600|
|      Apple|          300|
|    Generic|         1400|
+-----------+-------------+



In [262]:
df_Devices_Actions = df_D3.withColumn('All_Actions', F.sum('Count_Actions').over(Window.partitionBy()))\
            .withColumn('Ratio_Actions', F.col('Count_Actions') / F.col('All_Actions'))\
            .select('Name_Device', 'Count_Actions', 'Ratio_Actions')
       
df_Devices_Actions.show()

22/12/30 12:35:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------------+-------------------+
|Name_Device|Count_Actions|      Ratio_Actions|
+-----------+-------------+-------------------+
|    Generic|         1400|0.15384615384615385|
|     Spider|         3800| 0.4175824175824176|
|      Apple|          300|0.03296703296703297|
|     Huawei|         1100|0.12087912087912088|
|    Samsung|         1900| 0.2087912087912088|
|        HTC|          600|0.06593406593406594|
+-----------+-------------+-------------------+

